# Midterm Exam Big Data part 2
- Name : 
- NRP : 

2. (RecommendaƟon Systems – Frequent Patern Mining)

You are given the market basket dataset (market-basket.csv).

Use the FP-growth algorithm in Apache Spark to find the most frequent items. 

Save your result into an .xlsx files.

Do the experiment with different minSupport and minConfidence values. Give the
conclusion at the end of experiment.


## 2. Data Loading

In [1]:
# Install Apache Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=6ad59f2a7707049d3372b98e09ac7a119613a5d7635978c6d6127023f33f0819
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# Import required libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.fpm import FPGrowth


In [3]:
# Create Spark Session/Context
spark = SparkSession.builder \
    .master("local") \
    .appName("Frequent Itemset") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [11]:
# Import the data from CSV
market = spark.read.option("header", True).option("delimiter",";").csv("market-basket.csv")
market.show()

+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|SET 7 BABUSHKA NE...|       2|01.12.2010 08:26| 7,65|     17850|United Kingdom|
|536365|GLASS STAR FROSTE...|       6|01.12.2010 08:26| 4,25|     17850|United Kingdom|
|536366|HAND WARMER UNION...|       6|01.12.2010 08:28| 1,85|     17850|United Kingdom|
|536366|HAND WARMER RED P...|   

## 3. Data Preprocessing

We need to prepare the data into combination of ["id", "items"] = [1, [item1, item2, item3]]

In [12]:
# Select only the first two column
market_raw = market.select("billNo", "Itemname")
market_raw.show()

+------+--------------------+
|billNo|            Itemname|
+------+--------------------+
|536365|WHITE HANGING HEA...|
|536365| WHITE METAL LANTERN|
|536365|CREAM CUPID HEART...|
|536365|KNITTED UNION FLA...|
|536365|RED WOOLLY HOTTIE...|
|536365|SET 7 BABUSHKA NE...|
|536365|GLASS STAR FROSTE...|
|536366|HAND WARMER UNION...|
|536366|HAND WARMER RED P...|
|536367|ASSORTED COLOUR B...|
|536367|POPPY'S PLAYHOUSE...|
|536367|POPPY'S PLAYHOUSE...|
|536367|FELTCRAFT PRINCES...|
|536367|IVORY KNITTED MUG...|
|536367|BOX OF 6 ASSORTED...|
|536367|BOX OF VINTAGE JI...|
|536367|BOX OF VINTAGE AL...|
|536367|HOME BUILDING BLO...|
|536367|LOVE BUILDING BLO...|
|536367|RECIPE BOX WITH M...|
+------+--------------------+
only showing top 20 rows



In [13]:
# Before dropping the duplicates
market_raw.count()

522064

In [17]:
# Remove the duplicates
market_raw = market_raw.dropDuplicates(["billNo", "Itemname"])
market_raw.count()

511280

In [27]:
# Convert into this format: ["billNo", "items"] = [1, [a, b, c]]

market_input = market_raw.groupBy("billNo").agg(collect_list('Itemname').alias('items'))
market_input.show()

+------+--------------------+
|billNo|               items|
+------+--------------------+
|536365|[KNITTED UNION FL...|
|536366|[HAND WARMER UNIO...|
|536367|[BOX OF VINTAGE J...|
|536368|[YELLOW COAT RACK...|
|536369|[BATH BUILDING BL...|
|536370|[SPACEBOY LUNCH B...|
|536371|[PAPER CHAIN KIT ...|
|536372|[HAND WARMER UNIO...|
|536373|[GLASS STAR FROST...|
|536374|[VICTORIAN SEWING...|
|536375|[SAVE THE PLANET ...|
|536376|[RED HANGING HEAR...|
|536377|[HAND WARMER RED ...|
|536378|[PACK OF 60 PINK ...|
|536380|[JAM MAKING SET P...|
|536381|[ZINC WILLIE WINK...|
|536382|[VINTAGE SNAKES &...|
|536384|[ENAMEL BREAD BIN...|
|536385|[TRADITIONAL CHRI...|
|536386|[JUMBO BAG RED RE...|
+------+--------------------+
only showing top 20 rows



## 4. FPGrowth Model

In [19]:
def fpGrowthModel(ms, mc):
  # 1. Create the instance
  fpGrowth = FPGrowth(itemsCol="items", minSupport=ms, minConfidence=mc)

  # 2. Train/fit the data training to become a model
  model = fpGrowth.fit(market_input)
  
  return model

In [36]:
# fpGrowth with minSupport=0.1, minConfidence=0.1
m1 = fpGrowthModel(0.1, 0.1)

# fpGrowth with minSupport=0.5, minConfidence=0.5
m2 = fpGrowthModel(0.05, 0.05)

# fpGrowth with minSupport=0.25, minConfidence=0.25
m3 = fpGrowthModel(0.05, 0.1)

## 5. Evaluation

Analyzing the obtained results

In [37]:
# Display Frequent Itemset and generated association rules.
def showResult(model) :
  model.freqItemsets.show()
  #model.associationRules.show()

In [38]:
showResult(m1)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[WHITE HANGING HE...|2202|
+--------------------+----+



In [39]:
showResult(m2)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[JAM MAKING SET P...|1116|
|[LUNCH BAG  BLACK...|1260|
|[HEART OF WICKER ...|1181|
|[JAM MAKING SET W...|1089|
|[REGENCY CAKESTAN...|1904|
|[ASSORTED COLOUR ...|1431|
|[LUNCH BAG SPACEB...|1147|
|[JUMBO SHOPPER VI...|1166|
|[LUNCH BAG RED RE...|1541|
|[PACK OF 72 RETRO...|1279|
|[LUNCH BAG CARS B...|1132|
|[JUMBO BAG PINK P...|1211|
|[JUMBO BAG RED RE...|2064|
|[WOODEN PICTURE F...|1092|
|[NATURAL SLATE HE...|1232|
|    [SPOTTY BUNTING]|1114|
|[JUMBO STORAGE BA...|1178|
|[RECIPE BOX PANTR...|1085|
|     [PARTY BUNTING]|1656|
|[PAPER CHAIN KIT ...|1143|
+--------------------+----+
only showing top 20 rows



In [40]:
showResult(m3)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[JAM MAKING SET P...|1116|
|[LUNCH BAG  BLACK...|1260|
|[HEART OF WICKER ...|1181|
|[JAM MAKING SET W...|1089|
|[REGENCY CAKESTAN...|1904|
|[ASSORTED COLOUR ...|1431|
|[LUNCH BAG SPACEB...|1147|
|[JUMBO SHOPPER VI...|1166|
|[LUNCH BAG RED RE...|1541|
|[PACK OF 72 RETRO...|1279|
|[LUNCH BAG CARS B...|1132|
|[JUMBO BAG PINK P...|1211|
|[JUMBO BAG RED RE...|2064|
|[WOODEN PICTURE F...|1092|
|[NATURAL SLATE HE...|1232|
|    [SPOTTY BUNTING]|1114|
|[JUMBO STORAGE BA...|1178|
|[RECIPE BOX PANTR...|1085|
|     [PARTY BUNTING]|1656|
|[PAPER CHAIN KIT ...|1143|
+--------------------+----+
only showing top 20 rows



### Best fpGrowthmodel hyperparameter is: minSupport=0.05, minConfidence=0.05

## 6. Saving data into .xlsx



In [47]:
m2_data = m2.freqItemsets

type(m2_data)

pandasDF = m2_data.toPandas()
pandasDF.to_excel('output.xlsx')